# Simulating production environment

In [ ]:
region = 'AE'

In [4]:
path_region = f'../data/regions/{region}'

## Data

In [6]:
import pandas as pd
import os

path_data = os.path.join(path_region, 'prod.csv')
df = pd.read_csv(path_data, index_col=0, parse_dates=True)

## Preprocessing

### Scale data

In [9]:
import joblib

scaler = joblib.load(f'../data/regions/{region}/scaler.pkl')

In [12]:
sequence = scaler.transform(df)

### Feature selection by windows

In [15]:
import sys
sys.path.append('../')

from us_energy_demmand_forecast import utils

In [17]:
X, y = utils.create_sequences(sequence, 24)

## Simulate production environment

### Load the model

In [18]:
import lightgbm as lgb

model = lgb.Booster(model_file=f'../data/regions/{region}/model.txt')

### Forecast

In [20]:
y_pred = model.predict(X[:,:,0])
y_pred

array([0.30372531, 0.29856293, 0.27855674, ..., 0.28216819, 0.27000093,
       0.25398941])

### Unscaled forecast

In [23]:
y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1))
y_true = scaler.inverse_transform(y.reshape(-1, 1))

#### Create DataFrame

In [24]:
dic_pred = {
    'y_pred': y_pred.flatten(),
    'y_true': y_true.flatten(),
}

df_pred = pd.DataFrame(dic_pred)
df_pred

,y_pred,y_true
0,1203.212922,1196.0
1,1191.277496,1101.0
...,...,...
6710,1125.242161,1049.0
6711,1088.223513,990.0


#### Evaluate Model

In [25]:
from sklearn.metrics import mean_squared_error

In [27]:
mean_squared_error(df_pred.y_true, df_pred.y_pred, squared=False)

147.1668009195589